<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
    
</div>


# Equivalent sites circle with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Plot of the fitting model](#Plot-of-the-fitting-model)
- [Creating the reference data](#Creating-the-reference-data)
- [Setting and fitting](#Setting-and-fitting)  
- [Plotting the results](#Plotting-the-results)  

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use the <b>Equivalent Sites Circle</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
Please note that the following units are used for the QENS models

| Type of parameter | Unit          |
| :---------------- | :-----------: |
| Time              | picosecond    |
| Length            | Angstrom      |
| Momentum transfer | 1/Angstrom    |

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "1/ps", 
                       'q': "1/Angstrom", 
                       'scale': "unit_of_signal/ps", 
                       'center': "1/ps", 
                       'radius': "Angstrom", 
                       'resTime': "ps"}

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np

# for interactivity (plots, buttons...)
import ipywidgets

import matplotlib.pyplot as plt

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = ipywidgets.Button(description='Yes', button_style='success')
    buttonN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install the QENSmodels' library?"), ipywidgets.HBox([buttonY, buttonN])],
    layout=ipywidgets.Layout(width='50%', height='80px'))
    display(choice_installation)
    
    def on_buttonY_clicked(b):
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels

    def on_buttonN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")
     
    buttonY.on_click(on_buttonY_clicked)
    buttonN.on_click(on_buttonN_clicked) 

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):   
    lmfitY = ipywidgets.Button(description='Yes', button_style='success')
    lmfitN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install lmfit?"), ipywidgets.HBox([lmfitY, lmfitN])],
    layout=ipywidgets.Layout(width='30%', height='80px'))

    display(choice_installation)
    
    def on_lmfitY_clicked(b):
        !{sys.executable} -m pip install lmfit
    def on_lmfitN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")
        
    lmfitY.on_click(on_lmfitY_clicked)
    lmfitN.on_click(on_lmfitN_clicked) 

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

# import model from QENS library
import QENSmodels

[Top](#Table-of-Contents)

## Plot of the fitting model

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
# Dictionary of initial values
ini_parameters = {'q': 1., 'scale': 5., 'center': 5., 'Nsites': 3, 'radius': 5., 'resTime': 1.} 

def interactive_fct(q, scale, center, Nsites, radius, resTime):
    xs = np.linspace(-10, 10, 100)
    
    fig1, ax1 = plt.subplots()
    ax1.plot(xs, QENSmodels.sqwEquivalentSitesCircle(xs, q, scale, center, Nsites, radius, resTime))
    ax1.set_xlabel('x')
    ax1.grid()

# Define sliders for modifiable parameters and their range of variations

q_slider = ipywidgets.FloatSlider(value=ini_parameters['q'],
                                  min=0.1, max=10., step=0.1,
                                  description='q', 
                                  continuous_update=False) 

scale_slider = ipywidgets.FloatSlider(value=ini_parameters['scale'],
                                      min=0.1, max=10, step=0.1,
                                      description='scale',
                                      continuous_update=False) 

center_slider = ipywidgets.IntSlider(value=ini_parameters['center'],
                                     min=-10, max=10, step=1,
                                     description='center', 
                                     continuous_update=False) 

Nsites_slider = ipywidgets.IntSlider(value=ini_parameters['Nsites'],
                                     min=2, max=10, step=1,
                                     description='Nsites',
                                     continuous_update=False)

radius_slider = ipywidgets.FloatSlider(value=ini_parameters['radius'],
                                       min=0.1, max=10, step=0.1,
                                       description='radius',
                                       continuous_update=False)

resTime_slider = ipywidgets.FloatSlider(value=ini_parameters['resTime'],
                                        min=0.1, max=10, step=0.1,
                                        description='resTime', 
                                        continuous_update=False)

grid_sliders = ipywidgets.HBox([ipywidgets.VBox([q_slider, scale_slider, center_slider])
                                ,ipywidgets.VBox([Nsites_slider, radius_slider, resTime_slider])])
                            

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    q_slider.value = ini_parameters['q'] 
    scale_slider.value = ini_parameters['scale'] 
    center_slider.value = ini_parameters['center']  
    Nsites_slider.value = ini_parameters['Nsites'] 
    radius_slider.value = ini_parameters['radius'] 
    resTime_slider.value = ini_parameters['resTime']

# Define reset button and occurring action when clicking on it
reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

# Display the interactive plot
interactive_plot = ipywidgets.interactive_output(interactive_fct,       
                                         {'q': q_slider, 
                                          'scale': scale_slider,
                                          'center': center_slider,
                                          'Nsites': Nsites_slider,
                                          'radius': radius_slider,
                                          'resTime': resTime_slider})  
                                            
display(grid_sliders, interactive_plot, reset_button)

[Top](#Table-of-Contents)

## Creating the reference data

In [ ]:
nb_points = 200
xx = np.linspace(-5, 5, nb_points)
added_noise = np.random.normal(0, 1, nb_points)

equiv_sites_circle_noisy = QENSmodels.sqwEquivalentSitesCircle(xx, 
                                                               q=1., 
                                                               scale=1.3, 
                                                               center=0.3, 
                                                               Nsites=5, 
                                                               radius=4., 
                                                               resTime=3.)*\
                           (1 + 0.1*added_noise) + 0.01*added_noise

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
gmodel = Model(QENSmodels.sqwEquivalentSitesCircle)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

ini_values = {'scale': 1.22, 'center': 0.2, 'Nsites': 5, 'radius': 3.1, 'resTime': 0.33}

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('radius', min=0)
gmodel.set_param_hint('resTime', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)
gmodel.set_param_hint('Nsites', vary=False)

# Fit
result = gmodel.fit(equiv_sites_circle_noisy, w=xx, q=1.,
                    scale=ini_values['scale'], 
                    center=ini_values['center'],
                    Nsites=ini_values['Nsites'],
                    radius=ini_values['radius'],
                    resTime=ini_values['resTime'])

In [ ]:
# Plots - Initial model and reference data
fig0, ax0 = plt.subplots()
ax0.plot(xx, equiv_sites_circle_noisy, 'b.-', label='reference data')
ax0.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax0.set(xlabel='x', title='Initial model and reference data')
ax0.grid()
ax0.legend();

[Top](#Table-of-Contents)

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting result using lmfit functionality
result.plot()

In [ ]:
# plot fitting resulting using matplotlib.pyplot
fig1, ax1 = plt.subplots()
ax1.plot(xx, equiv_sites_circle_noisy, 'b-', label='reference data')
ax1.plot(xx, result.best_fit, 'r', label='fitting result')
ax1.legend()
ax1.set(xlabel='x', title='Fit result and reference data')
ax1.grid();

In [ ]:
for item in ['resTime', 'radius', 'center', 'scale']:
    print(item, result.params[item].value, '+/-', result.params[item].stderr, dict_physical_units[item])